In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.functional as f 
from torch.autograd import Variable
torch.backends.cudnn.deterministic = True
import pandas as pd
import numpy as np
import torch.utils.data as data_utils
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

In [3]:
number = 1
fileTrain = '/home/franciscoperez/Documents/GitHub/data/BIASEDFATS/Train_rrlyr-'+str(number)+'.csv'
fileTest = '/home/franciscoperez/Documents/GitHub/data/BIASEDFATS/Test_rrlyr-'+str(number)+'.csv'
train_dataset = pd.read_csv(fileTrain, index_col ='Unnamed: 0')
test_dataset = pd.read_csv(fileTest)

In [4]:
try:
    train_dataset =  train_dataset.drop(['Pred', 'Pred2', 'h', 'e', 'u','ID'], axis = 1)
    test_dataset = test_dataset[list(train_dataset.columns)]
except:
    print('---')

In [5]:
n = train_dataset.shape[0] 
input_size = train_dataset.shape[1]-1
hidden_size = 100
hidden_size2 = 100
num_classes = 2
num_epochs = 50
batch_size = 32
learning_rate = 0.001
regularization = False
add_DR_based_data = True


In [6]:
minimum_period = train_dataset['PeriodLS'].min()
maximum_period = train_dataset['PeriodLS'].max()


train_dataset = train_dataset.sample(n)
label = train_dataset['label']
del train_dataset['label']
scaler = StandardScaler()
train_dataset_scaled = scaler.fit_transform(train_dataset)
train_dataset = pd.DataFrame(train_dataset_scaled, columns=train_dataset.columns)
train_dataset['label'] = label
train_dataset.head()


,Amplitude,AndersonDarling,Autocor_length,Beyond1Std,CAR_mean,CAR_sigma,CAR_tau,Con,Eta_e,FluxPercentileRatioMid20,...,Skew,SlottedA_length,SmallKurtosis,Std,StetsonK,StetsonK_AC,StructureFunction_index_21,StructureFunction_index_31,StructureFunction_index_32,label
0,-0.514987,0.107490,-0.284489,0.333715,-0.001869,0.816393,-0.033293,-0.238377,-0.070306,0.289010,...,0.054652,-0.096971,-0.261869,-0.488317,0.309904,0.248875,-0.842580,-0.714159,0.342209,ClassB
1,-0.412166,-0.353378,0.894954,0.333715,-0.001798,1.124202,-0.033620,-0.531536,-0.062992,0.252125,...,0.051096,-0.491377,-0.358583,-0.396467,0.450689,-0.398543,0.843398,0.885671,0.783394,ClassB
2,-0.572823,1.422167,-0.621473,-0.099947,-0.001881,-0.680380,-0.033083,-0.531536,-0.070461,0.201736,...,-0.135977,-0.294606,-0.054042,-0.546505,-0.290366,-0.201901,0.178371,0.185648,0.252165,ClassB
3,-0.470002,0.252973,-0.284489,0.220974,-0.001863,-0.510309,-0.033337,-0.531536,-0.062328,-0.195858,...,0.143167,-0.408632,-0.236068,-0.448973,0.108854,-0.443699,-0.356354,-0.466542,-0.910932,NaN
4,-0.482855,-0.338269,-0.284489,0.164587,-0.001877,1.536734,-0.033180,-0.238377,-0.071884,0.277867,...,-0.774369,-0.054457,-0.275064,-0.468088,0.182466,1.073983,1.116882,1.095356,0.617891,ClassB


In [7]:
label = test_dataset[['label']]
del test_dataset['label']
test_dataset_scaled = scaler.transform(test_dataset)
test_dataset = pd.DataFrame(test_dataset_scaled, columns=test_dataset.columns)
test_dataset['label'] = label
test_dataset.head()

,Amplitude,AndersonDarling,Autocor_length,Beyond1Std,CAR_mean,CAR_sigma,CAR_tau,Con,Eta_e,FluxPercentileRatioMid20,...,Skew,SlottedA_length,SmallKurtosis,Std,StetsonK,StetsonK_AC,StructureFunction_index_21,StructureFunction_index_31,StructureFunction_index_32,label
0,0.368630,-0.653029,-0.621473,-1.244811,-0.001754,-0.438519,-0.033638,-0.531536,-0.071938,-0.603972,...,1.321295,-0.345227,-0.169991,0.432532,-0.086576,1.829801,-0.118080,-0.112777,0.446760,ClassB
1,1.056245,-0.643994,-0.621473,1.292115,-0.001854,-0.474296,-0.033354,-0.531536,-0.070766,1.925675,...,-0.602614,-0.432620,-0.728697,1.093200,1.581364,1.247411,-0.402680,-0.319404,0.663064,ClassA
2,0.456992,-0.652955,-0.621473,1.417879,-0.001850,0.981196,-0.033373,-0.531536,-0.041288,-0.146607,...,-1.147137,-0.420964,-0.709481,0.640285,1.554462,1.321814,-1.269708,-1.238012,-1.546118,ClassB
3,-0.045866,-0.653029,0.557970,0.784730,-0.001780,1.432169,-0.033627,-0.238377,-0.072123,5.525472,...,0.205319,-0.489436,-0.702007,0.054557,2.397809,0.967167,1.162710,1.158074,0.714759,ClassB
4,-0.094064,-0.652326,-0.621473,-0.737426,-0.001851,0.762495,-0.033338,-0.531536,-0.071305,-0.309453,...,0.501169,-0.342244,0.012441,-0.150451,-0.070468,0.588588,0.018325,-0.107456,-0.452754,ClassB


In [8]:
(np.random.uniform(0.1,1.1)-minimum_period)/(maximum_period-minimum_period)

1.1205931855941424e-05

In [9]:
samples = 500
if add_DR_based_data:
    for i in range(samples):
        new_data = pd.DataFrame(train_dataset.sample(1000).mean()).T
        new_data['PeriodLS']= (np.random.uniform(0.1,1.1)-minimum_period)/(maximum_period-minimum_period)
        new_data['label'] = 'ClassA'
        frames = [train_dataset, new_data]
        train_dataset = pd.concat(frames)
    
    for i in range(0):
        new_data = pd.DataFrame(train_dataset.sample(1000).mean()).T
        new_data['PeriodLS']=(np.random.uniform(0.0,0.2)-minimum_period)/(maximum_period-minimum_period)
        new_data['label'] = 'ClassB'
        frames = [train_dataset, new_data]
        train_dataset = pd.concat(frames)
        
    for i in range(0):    
        new_data = pd.DataFrame(train_dataset.sample(1000).mean()).T
        new_data['PeriodLS']=(np.random.uniform(1.0,1.2)-minimum_period)/(maximum_period-minimum_period)
        new_data['label'] = 'ClassB'
        frames = [train_dataset, new_data]
        train_dataset = pd.concat(frames)

In [10]:
train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.1, random_state=42)

Cases using DR 1: Period $ \in [0.2,1.0]$ days

In [11]:
train_dataset['label'] = 1*(train_dataset['label'] == 'ClassA')
train_target = torch.tensor(train_dataset['label'].values.astype(np.float32))
train = torch.tensor(train_dataset.drop('label', axis = 1).values.astype(np.float32)) 
train_tensor = data_utils.TensorDataset(train, train_target) 
train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)

In [12]:
val_dataset['label'] = 1*(val_dataset['label'] == 'ClassA')
val_target = torch.tensor(val_dataset['label'].values.astype(np.float32))
val = torch.tensor(val_dataset.drop('label', axis = 1).values.astype(np.float32)) 
val_tensor = data_utils.TensorDataset(val, val_target) 
val_loader = data_utils.DataLoader(dataset = val_tensor, batch_size = batch_size, shuffle = True)

In [13]:
test_dataset['label'] = 1*(test_dataset['label'] == 'ClassA')
test_target = torch.tensor(test_dataset['label'].values.astype(np.float32))
test = torch.tensor(test_dataset.drop('label', axis = 1).values.astype(np.float32)) 
test_tensor = data_utils.TensorDataset(test, test_target) 
test_loader = data_utils.DataLoader(dataset = test_tensor, batch_size = batch_size, shuffle = True)

In [14]:
# Neural Network Model (1 hidden layer)
class Net(nn.Module):
    def __init__(self, input_size, hidden_size,hidden_size2, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size2)  
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out
    
net = Net(input_size, hidden_size, hidden_size2, num_classes)
net.cuda()   

Net(
  (fc1): Linear(in_features=63, out_features=100, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=100, out_features=2, bias=True)
)

In [15]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)  

In [17]:
# Train the Model   
for epoch in range(num_epochs):
    print('-----------Training----------------------------')
    epoch_loss = 0.0
    running_loss = 0.0
    for i, (star, labels) in enumerate(train_loader):  
        star = Variable(star.view(-1, input_size)).cuda()
        labels = Variable(labels).cuda()
        #print(star[:,49])
        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = net(star)
        #print(softmax(outputs))
        loss = criterion(outputs, labels.long())
        
        if regularization:
            l1_regularization = 0
            l2_regularization = 0
            lambda1 = 0.001
            lambda2 = 0.001
            for param in net.parameters():
                l1_regularization += torch.norm(param, 1)**2
                l2_regularization += torch.norm(param, 2)**2
            loss = loss + lambda1*l1_regularization + lambda2*l2_regularization
                
        loss.backward()
        optimizer.step()
        
        epoch_loss += outputs.shape[0] * loss.item()      
        running_loss += loss.item()
        
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    print(epoch+1, epoch_loss / len(train_loader))
    print('-----------Validation----------------------------')
    epoch_loss = 0.0
    running_loss = 0.0
    for i, (star, labels) in enumerate(val_loader):  
        star = Variable(star.view(-1, input_size)).cuda()
        labels = Variable(labels).cuda()
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = net(star)
        loss = criterion(outputs, labels.long())    
        epoch_loss += outputs.shape[0] * loss.item()      
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    print(epoch+1, epoch_loss / len(val_loader))

-----------Training----------------------------
[1,  2000] loss: 0.274
[1,  4000] loss: 0.268
[1,  6000] loss: 0.266
[1,  8000] loss: 0.267
[1, 10000] loss: 0.260
1 8.5493563162875
-----------Validation----------------------------
1 8.531878313757662
-----------Training----------------------------
[2,  2000] loss: 0.262
[2,  4000] loss: 0.262
[2,  6000] loss: 0.264
[2,  8000] loss: 0.264
[2, 10000] loss: 0.265
2 8.430997454563236
-----------Validation----------------------------
2 8.469251475365283
-----------Training----------------------------
[3,  2000] loss: 0.260
[3,  4000] loss: 0.265
[3,  6000] loss: 0.260
[3,  8000] loss: 0.265
[3, 10000] loss: 0.263
3 8.41225038568449
-----------Validation----------------------------
3 8.473161280835821
-----------Training----------------------------
[4,  2000] loss: 0.261
[4,  4000] loss: 0.267
[4,  6000] loss: 0.262
[4,  8000] loss: 0.267
[4, 10000] loss: 0.257
4 8.405464811238572
-----------Validation----------------------------
4 8.4713339

32 8.563760453394536
-----------Training----------------------------
[33,  2000] loss: 0.258
[33,  4000] loss: 0.260
[33,  6000] loss: 0.258
[33,  8000] loss: 0.263
[33, 10000] loss: 0.260
33 8.329866844953592
-----------Validation----------------------------
33 8.620896912612883
-----------Training----------------------------
[34,  2000] loss: 0.260
[34,  4000] loss: 0.261
[34,  6000] loss: 0.261
[34,  8000] loss: 0.262
[34, 10000] loss: 0.258
34 8.324492848433064
-----------Validation----------------------------
34 8.598546018899945
-----------Training----------------------------
[35,  2000] loss: 0.264
[35,  4000] loss: 0.257
[35,  6000] loss: 0.255
[35,  8000] loss: 0.260
[35, 10000] loss: 0.263
35 8.315586968636026
-----------Validation----------------------------
35 8.538482253742062
-----------Training----------------------------
[36,  2000] loss: 0.262
[36,  4000] loss: 0.260
[36,  6000] loss: 0.261
[36,  8000] loss: 0.261
[36, 10000] loss: 0.260
36 8.324623722165079
----------

In [ ]:
# Test the Model
correct = 0
total = 0
for star, labels in test_loader:
    images = Variable(star.view(-1, input_size)).cuda()
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels.long()).sum()
print('Accuracy of the network on test objects: %d %%' % (100 * correct / total))

In [ ]:
correct = 0
total = 0
for star, labels in train_loader:
    images = Variable(star.view(-1, input_size)).cuda()
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels.long()).sum()
print('Accuracy of the network on train objects: %d %%' % (100 * correct / total))

In [ ]:
# Save the Model
torch.save(net.state_dict(), 'model.pkl')

In [ ]:
def softmax(x): 
    return x.exp() / (x.exp().sum(-1)).unsqueeze(-1)